## Model 1.0
Powered by [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


====================================================================================================

## 1

import libraries \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file \
memory allocations to manage ghosts


====================================================================================================

In [6]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from math import ceil
from typing import Tuple, List, Dict
import numpy as np
import csv
import os
import sys
from abc import ABC, abstractmethod


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return int(runner.schedule.tick)

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")


if os.path.isdir(params["log_file_root"]+"."+str(rank)):
    os.system("rm -R "+params["log_file_root"]+"."+str(rank))  
os.makedirs(params["log_file_root"]+"."+str(rank)) 

#copy in the output folder the starting set of parameters
os.system("cp model1.yaml "+params["log_file_root"]+"."+str(rank)+"/")
os.system("cp firm-features.csv "+params["log_file_root"]+"."+str(rank)+"/")

if rank==0:
    i=0
    while os.path.isdir(params["log_file_root"]+"."+str(rankNum+i)):
        os.system("rm -R "+params["log_file_root"]+"."+str(rankNum+i))
        i+=1
    

    
#moves to the right folder (that you must create and initialize with a firm-features.csv file)
if not os.path.isdir(params["log_file_root"]+"."+str(rank)):
    print("There is no "+params["log_file_root"]+"."+str(rank) + " starting folder!")  
    sys.exit(0)
else: os.chdir(params["log_file_root"]+"."+str(rank))


        
#dentro a home/model1: "ls "+"../"+params["log_file_root"]+"."+str(rankNum+i))

#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T() #launches the timer

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc() #launches the cpu timer

agent_cache={} # dict with uid as keys and agents' tuples as values


===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [7]:
# built-here function to check whether at least one item in a list is != 0
def any(iterable):
    for element in iterable:
        if element != 0:
            return True
    return False
    
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, labor:int, capital:float, minOrderDuration:int,\
                 maxOrderDuration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 assetsUsefulLife: float, plannedMarkup: float, orderObservationFrequency: int, productionType: int,\
                 sectoralClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        self.labor=labor
        self.capital=capital
        self.capitalQ= 0
        self.unavailableLabor=0
        self.unavailableCapitalQ=0
        self.minOrderDuration=minOrderDuration
        self.maxOrderDuration=maxOrderDuration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.plannedMarkup=plannedMarkup
        self.orderObservationFrequency=orderObservationFrequency
        self.productionType=productionType
        self.sectoralClass=sectoralClass
        
        self.lostProduction=0
        self.inventories=0
        self.inProgressInventories=0
        self.appRepository=[] #aPP is aProductiveProcess
        
        self.profits=0
        self.revenues=0
        self.totalCosts=0
        self.totalCostOfLabor=0
        self.totalCostOfCapital=0
        self.addedValue=0
        self.initialInventories=0
        self.capitalQDynamic=0
        self.myBalancesheet=np.zeros((params['howManyCycles'], 20))

        self.movAvQuantitiesInEachPeriod=[]
        self.movAvDurations=[]
        
        self.productiveProcessIdGenerator=0
      
        self.theCentralPlanner=0

        self.Deposits = []
        self.Loans = []
        
        self.InterestsToLoans = 0
        
    def settingCapitalQ(self, investmentGoodPrices):
        #############pt temporary solution
        #we temporary use this vector with a unique position as there is only one investment good at the moment
        self.priceOfDurableProductiveGoodsPerUnit = investmentGoodPrices[0] #1 
        self.currentPriceOfDurableProductiveGoodsPerUnit = investmentGoodPrices[0] #1  # the price to be paid to acquire 
                                                                                    # new capital in term of quantity
            
        #pt TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP
        #############   underlying idea:
        #               the actual initial price of durable productive goods (per unit of quantity) must be
        #               consistent with the initial cost of production of the durable productive goods;
        #
        #               the recipe set the ratio K/L where K is expressed in value;
        #
        #               having a price we implicitly set the "quantity";
        #
        #               substitution costs will consider both the change of the quantity and of the price
        #               at which the firm will pay the new productive goods;
        #
        #               the used v. unused capital measures are calculated as addenda of the capital in quantity
        #
        #               the costOfCapital (ratio of interests or rents) will be applied to the current value
        #               of the capital, after calculating the changes in quantity and then in value (considering 
        #               changes in q. and their value using the price of the new acquisitions)
        #
        #               as it evolves over time, the mean price of durable productive goods is an idiosyncratic
        #               property of the firm
        #
        #               L productivity is expressed in quantity as orders are expressed in quantity 

        self.capitalQ=self.capital/self.priceOfDurableProductiveGoodsPerUnit


    # activated by the Model
    def estimatingInitialPricePerProdUnit(self):

        total =  (1/self.laborProductivity)*params['wage']
        total += (1/self.laborProductivity)*self.recipe*params['costOfCapital']/params['timeFraction']
        total += (1/self.laborProductivity)*self.recipe/(self.assetsUsefulLife * params['timeFraction']) 
        if params['usingMarkup']: total *= (1+self.plannedMarkup)
        total *= ((self.maxOrderDuration+self.minOrderDuration)/2)
        return total
    
        
    def dealingMovAvElements(self, freq, x, y):
        
        self.movAvQuantitiesInEachPeriod.append(x/y)
        if len(self.movAvQuantitiesInEachPeriod) > freq: self.movAvQuantitiesInEachPeriod.pop(0) 
            
        self.movAvDurations.append(y)
        if len(self.movAvDurations) > freq: self.movAvDurations.pop(0)

        
    def receivingNewOrder(self, productionOrder: float, orderDuration):

        #creates a statistics of the values of the received order
        self.dealingMovAvElements(self.orderObservationFrequency, productionOrder, orderDuration)
        
        #decision on accepting or refusing the new order
        productionOrderQuantityByPeriod=productionOrder/orderDuration
        requiredLabor=np.ceil(productionOrderQuantityByPeriod/self.laborProductivity)
        requiredCapitalQ=requiredLabor*self.recipe/self.priceOfDurableProductiveGoodsPerUnit
        #if self.uid[0]==29 and self.uid[2]==2:
        #    print("***1",t(),"new order q. per period", productionOrderQuantityByPeriod,\
        #          "req L", requiredLabor, "L", self.labor)
        
        #create a new aPP or skip the order
        if requiredLabor <= self.labor and requiredCapitalQ <= self.capitalQ: 
            self.productiveProcessIdGenerator += 1
            productiveProcessId=(self.uid[0],self.uid[1],self.uid[2],self.productiveProcessIdGenerator)
            aProductiveProcess = ProductiveProcess(productiveProcessId,productionOrderQuantityByPeriod, \
                                                   requiredLabor, requiredCapitalQ, orderDuration,\
                                                   self.priceOfDurableProductiveGoodsPerUnit,\
                                                   self.assetsUsefulLife)
            self.appRepository.append(aProductiveProcess)


    def produce(self)->tuple: 
        
        #total values of the firm in the current interval unit
        self.currentTotalCostOfProductionOrder=0
        self.currentTotalOutput=0
        self.currentTotalCostOfUnusedFactors=0
        self.currentTotalLostProduction=0
        self.currentTotalCostOfLostProduction=0
        
        avgRequiredLabor=0
        avgRequiredCapitalQ=0
        
        if t()==0: self.initialInventories=0 
        else: self.initialInventories=self.inventories+self.inProgressInventories

        # activity within a time unit
        for aProductiveProcess in self.appRepository:  

            if not aProductiveProcess.hasResources and \
                        (self.labor - self.unavailableLabor >= aProductiveProcess.requiredLabor and\
                         self.capitalQ - self.unavailableCapitalQ >= aProductiveProcess.requiredCapitalQ):
                self.unavailableLabor += aProductiveProcess.requiredLabor
                self.unavailableCapitalQ += aProductiveProcess.requiredCapitalQ
                aProductiveProcess.hasResources = True 
                    
            if aProductiveProcess.hasResources: #resources may be just assigned above
                #production
                (aPPoutputOfThePeriod, aPPrequiredLabor, aPPrequiredCapitalQ, aPPlostProduction,\
                 aPPcostOfLostProduction) = aProductiveProcess.step()
                     
                self.currentTotalOutput += aPPoutputOfThePeriod
                
                cost = aPPrequiredLabor*params['wage'] \
                       + aPPrequiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit \
                                                         *params['costOfCapital']/params['timeFraction']\
                       + aPPrequiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit/ \
                         (self.assetsUsefulLife * params['timeFraction'])             
                                                       
                self.currentTotalCostOfProductionOrder += cost
                
                self.currentTotalLostProduction += aPPlostProduction
                self.currentTotalCostOfLostProduction += aPPcostOfLostProduction               
        
                if not params['usingMarkup']: self.plannedMarkup=0
                if aProductiveProcess.failure:
                    #consider markup
                    self.inProgressInventories -= cost*(aProductiveProcess.productionClock-1)*(1+self.plannedMarkup)
                    
                    #NB this is an approximation because in multiperiodal production processes the
                    #   priceOfDurableProductiveGoodsPerUnit may change, but it is a realistic
                    #   approximation in firm accounting               
                    
                else:
                    if aProductiveProcess.productionClock < aProductiveProcess.orderDuration:
                        self.inProgressInventories += cost * (1+self.plannedMarkup) #consider markup
                    else:
                        self.inventories+=cost*aProductiveProcess.orderDuration*(1+self.plannedMarkup)
                        self.inProgressInventories -= cost*(aProductiveProcess.orderDuration-1) *(1+self.plannedMarkup)
                        #consider markup (it is added in the final and subtracted by the inProgress)

        self.currentTotalCostOfUnusedFactors =  (self.labor - self.unavailableLabor)*params['wage'] + \
                                        (self.capitalQ - self.unavailableCapitalQ)*\
                                         self.priceOfDurableProductiveGoodsPerUnit*\
                                         params['costOfCapital']/params['timeFraction'] + \
                                         (self.capitalQ - self.unavailableCapitalQ) *\
                                            self.priceOfDurableProductiveGoodsPerUnit/ \
                                            (self.assetsUsefulLife * params['timeFraction'])
                                         # considering substitutions also for the idle capital
        
        #print("ORDER MOV AV",self.uid, sum(self.movAvQuantitiesInEachPeriod)/ len(self.movAvQuantitiesInEachPeriod), flush=True)
        avgRequiredLabor=np.ceil( ((sum(self.movAvQuantitiesInEachPeriod)/len(self.movAvQuantitiesInEachPeriod)) /self.laborProductivity )\
                *( sum(self.movAvDurations)/ len(self.movAvDurations) ))
        
        #total cost of labor
        self.totalCostOfLabor= self.labor*params['wage']
        
        #labor adjustments (frequency at orderObservationFrequency)
        if t() % self.orderObservationFrequency == 0 and t() > 0:
            if self.labor > (1+params['tollerance']) * avgRequiredLabor:
                self.labor = np.ceil((1+params['tollerance']) * avgRequiredLabor) #max accepted q. of L (firing)
            if self.labor < (1/(1+params['tollerance'])) * avgRequiredLabor:
                self.labor = np.ceil((1/(1+params['tollerance'])) * avgRequiredLabor) #min accepted q. of L (hiring)
            #if self.uid==(32,0,0): print("***",self.uid, "labM",avgRequiredLabor,"L", self.labor, flush=True)
           
        
        #capital adjustments (frequency at each cycle)
        #here the following variables are disambiguated between actual and desired values, so they appear in a double shape:
        # i) capital and capitalQ, ii) desiredCapitalSubstistutions and desiredCapitalQsubstitutions
        
        self.capitalBeforeAdjustment=self.capital
        desiredCapitalQsubstitutions=0
        desiredCapitalSubstitutions=0
        requiredCapitalQincrement=0
        requiredCapitalIncrement=0
        
        if t() > self.orderObservationFrequency: #no corrections before the end of the first correction interval
                                                 #where orders are under the standard flow of the firm
            capitalQmin= self.capitalQ/(1+params['tollerance'])
            capitalQmax= self.capitalQ*(1+params['tollerance'])
            
            avgRequiredCapital=avgRequiredLabor*self.recipe
            avgRequiredCapitalQ=avgRequiredCapital/self.currentPriceOfDurableProductiveGoodsPerUnit
            
            requiredCapitalSubstitution=self.capital/(self.assetsUsefulLife * params['timeFraction'])
            requiredCapitalSubstitutionQ=self.capitalQ/(self.assetsUsefulLife * params['timeFraction']) 
            
            #obsolescence  and deterioration effect
            self.capitalQ-=requiredCapitalSubstitutionQ
            self.capital-=requiredCapitalSubstitution
            
            a=(-requiredCapitalSubstitutionQ)
            #A=(-requiredCapitalSubstitution)
            
            #case I
            if avgRequiredCapitalQ < capitalQmin:
                b=avgRequiredCapitalQ-capitalQmin #being b<0
                #quantities
                if b<=a: desiredCapitalQsubstitutions=0
                if b>a: desiredCapitalQsubstitutions=abs(a)-abs(b)

                #values
                desiredCapitalSubstitutions=desiredCapitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
            
            #case II
            if capitalQmin <= avgRequiredCapitalQ and avgRequiredCapitalQ <= capitalQmax:
                #quantities
                desiredCapitalQsubstitutions=abs(a) 
    
                #values
                desiredCapitalSubstitutions=desiredCapitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
            
            #case III
            if avgRequiredCapitalQ > capitalQmax:
                #quantities
                desiredCapitalQsubstitutions=abs(a)
                requiredCapitalQincrement=avgRequiredCapitalQ-capitalQmax

                #values
                desiredCapitalSubstitutions=desiredCapitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
                requiredCapitalIncrement=requiredCapitalQincrement*self.currentPriceOfDurableProductiveGoodsPerUnit
        
        
        self.desiredCapitalQsubstitutions=desiredCapitalQsubstitutions
        self.requiredCapitalQincrement=requiredCapitalQincrement                
        self.desiredCapitalSubstitutions=desiredCapitalSubstitutions
        self.requiredCapitalIncrement=requiredCapitalIncrement

        
        #=========================================================================================
            # the key local variables are:
            #
            # desiredCapitalQsubstitutions & desiredCapitalSubstitutions
            #
            # requiredCapitalQincrement & requiredCapitalIncrement
            #
            #
            # giving:
            #
            # self.capitalQ increment (with +=) from desiredCapitalQsubstitutions + requiredCapitalQincrement
            #
            # self.capital increment (with +=) from desiredCapitalSubstitutions + requiredCapitalIncrement
            #
            # self.capitalQDynamic = desiredCapitalQsubstitutions+requiredCapitalQincrement, for reporting reasons
            #
            #
            # with return:
            #
            # self.capital
            # self.capitalQDynamic
            #=========================================================================================

    def allowInformationToCentralPlanner(self) -> tuple:
        return(self.desiredCapitalQsubstitutions, self.requiredCapitalQincrement,\
               self.desiredCapitalSubstitutions, self.requiredCapitalIncrement)
    
    
    def requestGoodsToTheCentralPlanner(self) -> tuple:
        return(self.uid, self.desiredCapitalQsubstitutions,self.requiredCapitalQincrement,\
                           self.desiredCapitalSubstitutions, self.requiredCapitalIncrement)
    
    
    def concludeProduction(self):
        
        #action of the planner
        capitalQsubstitutions = self.investmentGoodsGivenByThePlanner[0]
        capitalQincrement = self.investmentGoodsGivenByThePlanner[1]
        capitalSubstitutions = self.investmentGoodsGivenByThePlanner[2]
        capitalIncrement = self.investmentGoodsGivenByThePlanner[3]
        
        
        #effects
        self.capitalQ+=capitalQsubstitutions+capitalQincrement 
        self.capital+=capitalSubstitutions+capitalIncrement
        self.capitalQDynamic=capitalQsubstitutions+capitalQincrement

        
        
        #total cost of capital
        self.totalCostOfCapital=self.capitalBeforeAdjustment*params['costOfCapital']/params['timeFraction']\
                                +capitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
           

        # remove concluded aPPs from the list (backward to avoid skipping when deleting)
        for i in range(len(self.appRepository)-1,-1,-1):
            if self.appRepository[i].productionClock == self.appRepository[i].orderDuration: 
                self.unavailableLabor-=self.appRepository[i].requiredLabor
                self.unavailableCapitalQ-=self.appRepository[i].requiredCapitalQ
                del self.appRepository[i]
        
        return(self.currentTotalOutput, self.currentTotalCostOfProductionOrder, self.currentTotalCostOfUnusedFactors,self.inventories,\
               self.inProgressInventories, self.currentTotalLostProduction, self.currentTotalCostOfLostProduction, \
               self.labor, self.capital, self.capitalQDynamic)
               # labor, capital modified just above
        
    # TODO: transfer money from centralPlanner to firm
    def receiveSellingOrders(self, shareOfInventoriesBeingSold: float, centralPlannerBuyingPriceCoefficient: float):
        nominalQuantitySold=shareOfInventoriesBeingSold*self.inventories
        self.revenues=centralPlannerBuyingPriceCoefficient*nominalQuantitySold
        self.inventories-=nominalQuantitySold    

    # we may need to store the payment information inside the attribute
    # for the calculation about desireCalpital
    def payInterests(self):
        self.InterestsToLoans = 0
        for loan in self.Loans:
            timeDiff = loan.stratTick - t()
            if timeDiff % loan.period == 0 and timeDiff > 0:

                interest = loan.interestRate * loan.amount
                principal = loan.iniAmount/loan.length

                self.InterestsToAdvances += interest

                
                self.Reserves[0] -= interest + principal
                loan.amount -= principal

                # TODO remember to remove the expire or filled loan in the schedual

        
    def makeBalancesheet(self):
        self.totalCosts= self.currentTotalCostOfProductionOrder + self.currentTotalCostOfUnusedFactors
        """
        if params['usingMarkup']:
            self.inventories *= (1+self.plannedMarkup) #planned because != ex post
            self.inProgressInventories *= (1+self.plannedMarkup) 
        """
        
        self.profits= self.revenues+(self.inventories + self.inProgressInventories)\
                    -self.totalCosts-self.initialInventories 
        self.addedValue=self.profits+self.totalCosts
        
        self.myBalancesheet[t(), 0]=self.sectoralClass #i.e. row number in firms-features
        
        self.myBalancesheet[t(), 1]=self.initialInventories
        self.myBalancesheet[t(), 2]=self.totalCosts
        
        if not self.productionType in params["investmentGoods"]: self.myBalancesheet[t(), 3]=self.revenues
        else: self.myBalancesheet[t(), 4]=self.revenues

        if not self.productionType in params["investmentGoods"]: self.myBalancesheet[t(), 5]=self.inventories
        else: self.myBalancesheet[t(), 6]=self.inventories 
            
        if not self.productionType in params["investmentGoods"]: self.myBalancesheet[t(), 7]=self.inProgressInventories
        else: self.myBalancesheet[t(), 8]=self.inProgressInventories
        
        self.myBalancesheet[t(), 9]=self.profits
        self.myBalancesheet[t(), 10]=self.addedValue
        self.myBalancesheet[t(), 11]=self.currentTotalOutput
        self.myBalancesheet[t(), 12]=self.currentTotalCostOfProductionOrder
        self.myBalancesheet[t(), 13]=self.currentTotalCostOfUnusedFactors
        self.myBalancesheet[t(), 14]=self.currentTotalLostProduction
        self.myBalancesheet[t(), 15]=self.currentTotalCostOfLostProduction
        self.myBalancesheet[t(), 16]=self.totalCostOfLabor
        self.myBalancesheet[t(), 17]=self.totalCostOfCapital
        self.myBalancesheet[t(), 18]=self.capitalQDynamic
        self.myBalancesheet[t(), 19]=self.productionType
        
        
    
###########################################################################################################################


class ProductiveProcess(): # no core.Agent???
    def __init__(self, productiveProcessId: tuple, targetProductionOfThePeriod:float, requiredLabor:int,\
                 requiredCapitalQ:float, orderDuration:int, priceOfDurableProductiveGoodsPerUnit:float,\
                 assetsUsefulLife:float):
        
        self.targetProductionOfThePeriod=targetProductionOfThePeriod
        self.requiredLabor = requiredLabor
        self.requiredCapitalQ = requiredCapitalQ
        self.orderDuration = orderDuration
        self.productionClock=0
        self.hasResources= False
        self.productiveProcessId=productiveProcessId
        self.priceOfDurableProductiveGoodsPerUnit=priceOfDurableProductiveGoodsPerUnit
        self.assetsUsefulLife=assetsUsefulLife
        
    #def step(self, productionOrder)->tuple:
    def step(self)->tuple:
        
        lostProduction=0
        costOfLostProduction=0
        self.productionClock += 1
        self.failure=False
        
        # production failure
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.failure=True
            #if self.productiveProcessId[0]==29 and self.productiveProcessId[2]==2:
            #    print("***2",t(),"failure")
            #print("failure",flush=True)
            lostProduction=self.targetProductionOfThePeriod*self.productionClock
            self.targetProductionOfThePeriod=0
            costOfLostProduction=(params['wage']* self.requiredLabor+\
                                       (params['costOfCapital']/params['timeFraction'])* self.requiredCapitalQ*\
                                        self.priceOfDurableProductiveGoodsPerUnit)*self.productionClock+\
                                        (self.requiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit)/ \
                                        (self.assetsUsefulLife * params['timeFraction']) 
            self.orderDuration = self.productionClock   

        return(self.targetProductionOfThePeriod, self.requiredLabor, self.requiredCapitalQ, \
               lostProduction, costOfLostProduction)

    
############################################################################################################################
############################################################################################################################

    
from plannerMethods import *


class CentralPlanner(core.Agent):

    TYPE = 1
    
    def __init__(self, local_id: 0, rank: 0):
        super().__init__(id=local_id, type=CentralPlanner.TYPE, rank=rank) #uid
    
        self.zeroIncrementAndSubstitutions=False
        self.informationTable=np.zeros((params['howManyCycles']-1, 4))
        
        self.allFirmsDesiredCapitalQsubstitutions = 0
        self.allFirmsRequiredCapitalQincrement = 0 
        self.allFirmsDesiredCapitalSubstitutions = 0
        self.allFirmsRequiredCapitalIncrement = 0

        self.Deposits = []
        

    def decidingActions(self, model):
        
        #getting information
        if t()>0:
            self.informationTable[t()-1,0]=sum(model.totalInvGoodsRevenues[t()-1])
            self.informationTable[t()-1,1]=sum(model.totalInvGoodsInventories[t()-1])
            self.informationTable[t()-1,2]=sum(model.totalCapitalQDynamic[t()-1])
            currentPrice=context.agent((0, 0, rank)).currentPriceOfDurableProductiveGoodsPerUnit
            self.informationTable[t()-1,3]=sum(model.totalCapitalQDynamic[t()-1])*currentPrice  
        
        #making decisions
        self.zeroIncrementAndSubstitutions=zeroIncrementAndSubstitutions
    
    
    def diffusingProductionOrders(self):
        for aFirm in context.agents(agent_type=0):
            aFirm.receivingNewOrder(rng.random()*aFirm.maxOrderProduction,\
                    rng.integers(aFirm.minOrderDuration, aFirm.maxOrderDuration+1))
            
    
    def generateDemandOrders(self): # planner buying from firms
        #the central planner asks to firm a certain quantity of goods
        #we observe the outcome of this in the firms revenues
        
        for aFirm in context.agents(agent_type=0):
            shareOfInventoriesBeingSold=params['minOfInventoriesBeingSold']\
                                        + rng.random()*params['rangeOfInventoriesBeingSold']
            centralPlannerBuyingPriceCoefficient = params['centralPlannerPriceCoefficient'] #0.8 + rng.random()*0.4
            aFirm.receiveSellingOrders(shareOfInventoriesBeingSold, centralPlannerBuyingPriceCoefficient)
            
    def askFirmsInvGoodsDemand(self):
        for aFirm in context.agents(agent_type=0):
            (desiredCapitalQsubstitutions,requiredCapitalQincrement,\
                desiredCapitalSubstitutions,requiredCapitalIncrement) = aFirm.allowInformationToCentralPlanner()
            
            
        self.allFirmsDesiredCapitalQsubstitutions += desiredCapitalQsubstitutions
        self.allFirmsRequiredCapitalQincrement += requiredCapitalQincrement 
        self.allFirmsDesiredCapitalSubstitutions += desiredCapitalSubstitutions
        self.allFirmsRequiredCapitalIncrement += requiredCapitalIncrement
        # we re-set the sums at zero in the model
        # here we call them by using self, so that the model can record them
        
            
    def executeInvestmentGoodsDemandFromFirms(self):
        for aFirm in context.agents(agent_type=0):
            (aFirmUid, desiredCapitalQsubstitutions, requiredCapitalQincrement,\
             desiredCapitalSubstitutions, requiredCapitalIncrement) = aFirm.requestGoodsToTheCentralPlanner()
            
            #initial basic behavior (give all vs give zero)
            #to be improved by using functions in plannerMethods.py that can use the firms info through thier uid
    
            #give all 
            capitalQsubstitutions=desiredCapitalQsubstitutions
            capitalQincrement=requiredCapitalQincrement 
            capitalSubstitutions=desiredCapitalSubstitutions
            capitalIncrement=requiredCapitalIncrement
        
            #give zero
            if zeroIncrementAndSubstitutions:
                capitalQsubstitutions=0
                capitalQincrement=0 
                capitalSubstitutions=0
                capitalIncrement=0 
                
                
            aFirm.investmentGoodsGivenByThePlanner = (capitalQsubstitutions, capitalQincrement,\
                                                         capitalSubstitutions, capitalIncrement)


"""
In the Financial System model, we aim to achieve two main objectives:
    1. Extensibility: The structure of our model should be easily extended to support many-to-many relationships between clients and banks.
       In other words, the structure should be general enough to accommodate multiple clients interacting with multiple banks.
    2. Flexibility: The model should be flexible enough to support the inclusion of a market random matching mechanism in the future.
       This means that the structure should not be overly encapsulated, allowing for easy integration of new features.

To achieve both objectives, we have made changes to the previous bank structure.
I have changed the previous bank structure.

Why I need to change?
    In the previous version, we attempted to store all information inside the banks, but this approach only works in a one-to-one bank system.
    In a one-to-one system, the client's financial information can be stored within a single bank, and so the client only needs to store the bank instance.
    However, in a many-to-many bank system, clients may obtain loans from different banks, and banks may provide loans to different clients.
    In this case, we need to store the information locally within each agent.

According to the banchmark model in https://github.com/S120/jmab. we may take the full stock structure or part of it:
    Inside the Banchmark model solution: 
        1.Create a stockMatrix in every agent to Stock all Stock item like (Deposit, Loan, Goods, Bond)

        The structure of the stock matrix is the following:

		this.numberStocks = numberStocks;    ---------- the number of stock's types defind in the config file
		this.stockMatrix= new ArrayList[2];
		this.stockMatrix[0]=new ArrayList<List<Item>> ();  ----------  asset
		this.stockMatrix[1]=new ArrayList<List<Item>> ();  ----------  liability
		for (int i=0; i<this.numberStocks;i++){
			this.stockMatrix[0].add(new ArrayList<Item>());  -----like (Deposit, Loan, Goods, Bond)
			this.stockMatrix[1].add(new ArrayList<Item>());  -----like (Deposit, Loan, Goods, Bond)

        2.Create a Class for each type of Stock item. Example structure like: 
        Loan(amount, iniAmount, interestRate, startTime, Period, Length, assetHolder, liabilityHolder)
        
        In addition. the stockMatrix contain all stock in the system whatever you have or not and in both asset and liability. 
            Detailed information still in the class attributes like addValue, costOfLabour ...

        Advantage:
            1. The stockMatrix can be easily translated and combined to generate a global balance sheet.
            2. Using classes to store stock items avoids duplicate elements inside the assetHolder and liabilityHolder, making the code cleaner.
              (Space cost is equal to tuples in Python if you use __slot__).
            3. It provides a clear view of the stock of assets and liabilities within an agent.
        
        Disadvantage:
            1. Each agent will store some useless stocks, which may waste some space.

        Neutrally:
            Highily encapsulation and modularization: they build up a structured object oriented project,
            The model is extensible and customizable but 
            The model also harder to understand, modify, and maintain.

    
    Simplify model:
        we Keep using Class to storage the stocks. In this case, it is easy to extent to a complicate system.
        The stock Class only storage attributes, reduce the cost of space
        we will not use a stock matrix in early step. it make the model easy to modify and clear in current stage.
            In the other side, a stock element is not possible to be both asset and liablilty.

    


"""
#  here is some storage structure, to avoid duplicate record in each client, we shoule not set any function inside for minimum space,
#  __slots__ will limit the attribute number inside the class, also the cost of space same with a tuple.

class Deposit:
    __slots__ = ['amount', 'assetHolder', 'liabilityHolder']

    def __init__(self, amount, assetHolder, liabilityHolder):
        self.amount = amount
        self.assetHolder = assetHolder
        self.liabilityHolder = liabilityHolder

'''
stratTick : the create time t()
period : repayment period
length : the loan duration 
'''
class Loan:
    __slots__ = ['amount', 'iniAmount', 'interestRate', 'stratTick', 'period', 'length', 'payMode' ,'assetHolder', 'liabilityHolder']

    def __init__(self, amount, interestRate, stratTick, period, length, payMode, assetHolder, liabilityHolder):
        self.amount = amount
        self.iniAmount = iniAmount
        self.interestRate = interestRate
        self.stratTick = stratTick
        self.period = period
        self.length = length
        self.payMode = payMode
        self.assetHolder = assetHolder
        self.liabilityHolder = liabilityHolder

class Reserve:
    __slots__ = ['amount', 'assetHolder', 'liabilityHolder']

    def __init__(self, amount, assetHolder, liabilityHolder):
        self.amount = amount
        self.assetHolder = assetHolder
        self.liabilityHolder = liabilityHolder


class CommercialBank(core.Agent):
    TYPE = 2
    
    def __init__(self, local_id: int, rank: int, depositInterestRate: float):
        super().__init__(id=local_id, type=CommercialBank.TYPE, rank=rank)

        # params
        self.depositInterestRate = 0
        self.loanInterestRate = 0

        # stock attributes
        self.Deposits = []
        self.Loans = []
        self.Reserves = []
        # Currentlly, advances just for display and explain
        self.Advances = []

        # flow attributes
        self.InterestsToAdvances = 0
        self.InterestsToDeposits = 0

    def transfer(self, sourceDeposit, targetDeposit, amount):
        sourceBankRserve = sourceDeposit.liabilityHolder.Reserves[0]
        otherBankReserve = targetDeposit.liabilityHolder.Reserves[0]

        
        sourceDeposit.amount -= amount
        sourceBankRserve.amount -= amount

        targetDeposit.amount -= amount
        otherBankRserve.amount -= amount


    def payInterests(self):
        self.InterestsToAdvances = 0
        for loan in self.Advances:
            timeDiff = loan.stratTick - t()
            if timeDiff % loan.period == 0 and timeDiff > 0:

                interest = loan.interestRate * loan.amount
                principal = loan.iniAmount/loan.length

                self.InterestsToAdvances += interest

                
                self.Reserves[0] -= interest + principal
                loan.amount -= principal

                # TODO remember to remove the expire or filled loan in the schedual

            
    def payDepositInterests(self):
        self.InterestsToDeposits = 0
        for deposit in self.Deposits:
            interest = deposit.amount * self.depositInterestRate
            client = deposit.assetHolder
            client.interestOnDeposits += interest
            self.InterestsToDeposits += interest
            deposit.amount += interest
    
    """
    this function only use for storing cash in to the bank, and we assume each agent can only deposit on one bank
    we assume every client will match a random bank at the beginning.
    so we have two situation:
        1: the client current bank is not equal to the current bank
            create a new Deposit
            add all client deposit to the new one
            add Deposit to the client and the bank
            remove the old deposit in the client and the old bank
        2: the client current bank is equal to the current bank
            add all client deposit to the old one
            remove the old deposit in the client and the old bank
    
    then we transfer the money from cash to deposit
    """
    # def askDeposit(self, aClient, amount):


    # def askLoan(self, aClient, desireAmount):
    # ['amount', 'interestRate', 'stratTick', 'period', 'length', 'payMode' ,'assetOwnerount', 'liabilityOwner']
        # loan = Loan(desireAmount, self.loanInterestRate, t(), 10, 5, None, self, aClient)
        # aClient.Loans.append(loan)
        # self.Loans.append(loan)

        # if aClient.Deposits[0].liabilityHolder != self:
        #     deposit = Deposit(desiredAmount, aClient, self)
        #     self.Deposits.append(deposit)
        #     aClient.Deposit.append(deposit)
        # else:
        #     aClient.Deposits[0].amount += desireAmount





    


===================================================================================================

## 3

the model

===================================================================================================

In [8]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInvGoodsRevenues=[]
        self.totalConsGoodsRevenues=[]
        self.totalInvGoodsInventories=[]
        self.totalConsGoodsInventories=[]
        self.totalInProgressInvGoodsInventories=[]
        self.totalInProgressConsGoodsInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        self.updatedLabor=[]
        self.updatedCapital=[]
        self.totalCapitalQDynamic=[]
        self.totalInvGoodsRevenues=[]
        self.totalConsGoodsRevenues=[]
        self.firmData={}
        self.theCentralPlanner=0

        
        #the context and the runner are created in step 1 
      
        runner.schedule_event(          0.0,     self.initInvestmentGoodPrices) 
        runner.schedule_event(          0.0,     self.initGhosts) 
        
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.05,  1, self.plannerDecidingActions)
        runner.schedule_repeating_event(0.06,  1, self.plannerDiffusingProductionOrders)
        runner.schedule_repeating_event(0.07,  1, self.firmsProducing)
        runner.schedule_repeating_event(0.08,  1, self.plannerPreparingAndMakingDistribution)
        runner.schedule_repeating_event(0.1,  1, self.firmsConcludingProduction)
        runner.schedule_repeating_event(0.11, 1, self.firmsMakingFinancialTransactionsRelatedToCosts)
        
        runner.schedule_repeating_event(0.2,  1, self.plannerGeneratingDemandOrders) #invGoods for next period investments
        runner.schedule_repeating_event(0.21, 1, self.firmsMakingFinancialTransactionsRelatedToRevenues)
        runner.schedule_repeating_event(0.3,  1, self.enterprisesMakingBalancesheet) #enterprises=firms+banks
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        # we assume we have two bank
        for i in range(2):
            context.add(CommercialBank(i, 0, params["depositInterestRate"])) #local_id=i, rank=0


        
        #importing csv file containing info about firms 
        #share of firms of that class, L min, L max, K min, K max, order duration min, order duration max, 
        #recipe, L prod, max order production, assets' useful life, planned markup, 
        #order observation frequency min, order observation frequency max, production type
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=-1
            k=0
            #for each record in firm-features.csv
            #share of firms of that class, L min, L max, K min, K max, order duration min, order duration max, recipe, 
            #L prod, max order production, assets' useful life, planned markup, order observation frequency min, 
            #order observation frequency max, production type
            for row in firmReader:
                print(row)
                if self.rowNumber>=0:
                    for i in range(int(row[0] * params['Firm.count'])// rankNum):
                        labor= rng.integers(row[1], row[2]+1) #+1 because integers exclude extremes
                        capital= row[3] + rng.random()*(row[4] -row[3])
                        minOrderDuration= row[5]
                        maxOrderDuration= row[6]
                        recipe= row[7] #K/L 
                        laborProductivity= row[8]
                        maxOrderProduction= row[9]
                        avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                        plannedMarkup=row[11]
                        orderObservationFrequency=rng.integers(row[12], row[13]+1)
                        productionType=int(row[14]) #productionType in firm-features.csv indicates the production of
                                                #investment goods if it is into the investmentGoods list in yaml
                        sectoralClass=int(self.rowNumber)
                        aFirm =Firm(k, rank, labor, capital, minOrderDuration, maxOrderDuration, recipe, laborProductivity,\
                                maxOrderProduction, avgAssetsUsefulLife, plannedMarkup, orderObservationFrequency, productionType,\
                                sectoralClass)

                        for aBank in ctx.agents(agent_type=2, shuffle=True):
                            deposit = Deposit(capital, aFirm, aBank)
                            aFirm.Deposits.append(deposit)
                            aBank.Deposits.append(deposit)

                        context.add(aFirm)
                        k += 1 # first element of the UID of the agents
                self.rowNumber += 1
        
        
        ####################################################################################################
        ################################## CREATE CENTRAL PLANNER AGENT ####################################
        ####################################################################################################
        if rank==0: 
            self.theCentralPlanner=CentralPlanner(0,0) #local_id=0, rank=0
            context.add(self.theCentralPlanner)
            #the Central Planner is an agent that we call by its id -> context.agent((0,1,0))
            #we will create a ghost of this in the other ranks and ask for the ghosts of firms in other ranks !=0
            for aFirm in context.agents(agent_type=0):
                aFirm.theCentralPlanner = context.agent((0,1,0))
        else:
            pass #need to assign the central planner ghost using cache memory by calling its uid = (0,1,0)
            
  
        
    #initialize investment good prices
    def initInvestmentGoodPrices(self):
        self.investmentGoodPrices=[0]*len(params['investmentGoods'])
        
        for anInvGoodType in range(len(params['investmentGoods'])):
            count=0
            for aFirm in context.agents(agent_type=0):
                if aFirm.productionType == params['investmentGoods'][anInvGoodType]:
                    self.investmentGoodPrices[anInvGoodType]+=aFirm.estimatingInitialPricePerProdUnit()
                    count+=1
            if count != 0: self.investmentGoodPrices[anInvGoodType]/=count
        
        if not any(self.investmentGoodPrices): 
            print("\nThere are no investment goods!")
            sys.exit(0)
        
        for aFirm in context.agents(agent_type=0):
            aFirm.settingCapitalQ(self.investmentGoodPrices)
            if aFirm.uid[0]==0: print("rank",rank,"Initial price of durable productive goods per unit",\
                                      aFirm.priceOfDurableProductiveGoodsPerUnit, flush=True) #as an info to the user
                
                
    #initialize ghosts by sending them in the ranks before starting the simulation 
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            

    def plannerDecidingActions(self): 
        context.agent((0,1,0)).decidingActions(self) # self here is the model instance


    # may be we need to transfer all capitalQ demand to capital demand and get loan from banks.
    # On the other word, capital equal to cash/deposit, capitalQ means amount of capital goods and we need to use capital to buy capitalQ
    def banksApplyLaonsToFirms(self):
        for aFirm in context.agents(agent_type=0):
            aFirm.myBank.applyLoan(aFirm, aFirm.desiredCapitalSubstitutions)
            
    def plannerDiffusingProductionOrders(self):
        context.agent((0,1,0)).diffusingProductionOrders()
        
    
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber)) #for each cycle adds a sub-list of lenght number of firm class types
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalInvGoodsInventories.append([0]*(self.rowNumber))
        self.totalInProgressInvGoodsInventories.append([0]*(self.rowNumber))
        self.totalConsGoodsInventories.append([0]*(self.rowNumber))
        self.totalInProgressConsGoodsInventories.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        self.totalCostOfLostProduction.append([0]*(self.rowNumber))
        self.updatedLabor.append([0]*(self.rowNumber))
        self.updatedCapital.append([0]*(self.rowNumber))
        self.totalCapitalQDynamic.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0): #SHUFFLE to make them acting in random order
            aFirm.produce()
            
            
            
    def plannerPreparingAndMakingDistribution(self):
        context.agent((0,1,0)).askFirmsInvGoodsDemand()
        context.agent((0,1,0)).executeInvestmentGoodsDemandFromFirms()
    
    def firmsConcludingProduction(self):
        for aFirm in context.agents(agent_type=0):
            
            tupleOfProductionResults = aFirm.concludeProduction()

            self.totalProduction[t()][aFirm.sectoralClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[t()][aFirm.sectoralClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[t()][aFirm.sectoralClass] += tupleOfProductionResults[2]
            
            if not aFirm.productionType in params["investmentGoods"]: 
                self.totalConsGoodsInventories[t()][aFirm.sectoralClass] += tupleOfProductionResults[3]
                self.totalInProgressConsGoodsInventories[t()][aFirm.sectoralClass] += tupleOfProductionResults[4]  
            else: 
                self.totalInvGoodsInventories[t()][aFirm.sectoralClass] += tupleOfProductionResults[3]
                self.totalInProgressInvGoodsInventories[t()][aFirm.sectoralClass] += tupleOfProductionResults[4]
            
            #here we will need to separate invGoods and consGoods inventories (and in progr inventories)
            #same for revenues, to be added here to the series
            self.totalLostProduction[t()][aFirm.sectoralClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[t()][aFirm.sectoralClass] += tupleOfProductionResults[6]
            self.updatedLabor[t()][aFirm.sectoralClass] += tupleOfProductionResults[7]
            self.updatedCapital[t()][aFirm.sectoralClass] += tupleOfProductionResults[8]
            self.totalCapitalQDynamic[t()][aFirm.sectoralClass] += tupleOfProductionResults[9]
    
    def firmsMakingFinancialTransactionsRelatedToCosts(self):
        # pay wage, pay loan, buy capital goods
        centralPlanner = context.agent((0,1,0))
        for aFirm in context.agents(agent_type=0):
            # codes below may become a function inside the classes
            # pay loan
            aFirm.payInterests()

            CapitalQdemaind = aFirm.getCapitalQdemaind
            price =  centralPlanner.getPriceOfDurableProductiveGoodsPerUnit(CapitalQdemaind)
            totalAmount = price * CapitalQdemaind
            # buy capital goods
            # 1. deposite transfer
            sourceDeposit = aFirm.Deposits[0]
            targetDeposit = centralPlanner.Deposits[0]
            sourceDeposit.liabilityHolder.transfer(sourceDeposit, targetDeposit, totalAmount)

            # 2. goods transfer
            centralPlanner.capitalQ -= quantity

            if we need a CapitalQ Object:
                CapitalGood = CapitalGood(totalAmount, quantity, price, duration, capitalLaborRatio, aFirm, centralPlanner)
                aFirm.capitalQ.append(CapitalGood)
            else:
                aFirm.capitalQ += CapitalQdemaind
            

    def plannerGeneratingDemandOrders(self):
        context.agent((0,1,0)).generateDemandOrders()
        
        
    def firmsMakingFinancialTransactionsRelatedToRevenues(self):
        # sell goods
        centralPlanner = context.agent((0,1,0))
        for aFirm in context.agents(agent_type=0):
            totalAmount = aFirm.revenues
            # 1. deposit transfer
            
            sourceDeposit = aFirm.Deposits[0]
            targetDeposit = centralPlanner.Deposits[0]
            
            sourceDeposit.liabilityHolder.transfer(sourceDeposit, targetDeposit, totalAmount)

            # 2. goods transfer
            aFirm.inventories -= quantity

            if we need a Good Object:
                ConsumptionGood = ConsumptionGood(totalAmount, quantity, price, duration, aFirm, centralPlanner)
                centralPlanner.inventories.append(ConsumptionGood)
            else:
                centralPlanner.inventories += quantity

        

    
    def enterprisesMakingBalancesheet(self):
        self.totalInvGoodsRevenues.append([0]*(self.rowNumber))
        self.totalConsGoodsRevenues.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0):
            aFirm.makeBalancesheet()
            self.totalConsGoodsRevenues[t()][aFirm.sectoralClass] += aFirm.myBalancesheet[t(), 3]
            self.totalInvGoodsRevenues[t()][aFirm.sectoralClass] += aFirm.myBalancesheet[t(), 4]
            
      
    
    
    #finish
    def finish(self):
        
        print("cpu time - calculating phase", Tc(), "rank", rank, flush=True)
        
        # infos for data_analysis*.ipynb
        with open('plotInfo.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum,\
                             context.size(agent_type_ids=[0])[0]))
        
        
        #series____________________________________________________
        
        names=["_total_production_","_total_cost_of_production_","_total_cost_of_unused_factors_",
               "_total_inv_goods_revenues_", "_total_cons_goods_revenues_", 
               "_total_inv_goods_inventories_","_total_in_progress_inv_goods_inventories_",
               "_total_cons_goods_inventories_","_total_in_progress_cons_goods_inventories_",
               "_total_lost_production_","_total_cost_of_lost_production_","_updatedLabor_","_updatedCapital_",\
              "total_capQ_dynamic_"]
        contents=[self.totalProduction,self.totalCostOfProduction,self.totalCostOfUnusedFactors,
                  self.totalInvGoodsRevenues, self.totalConsGoodsRevenues, 
                  self.totalInvGoodsInventories,self.totalInProgressInvGoodsInventories,
                  self.totalConsGoodsInventories,self.totalInProgressConsGoodsInventories,
                  self.totalLostProduction,self.totalCostOfLostProduction,
                  self.updatedLabor,self.updatedCapital, self.totalCapitalQDynamic]
        
        for s in range(len(names)):
            with open(params["log_file_root"]+names[s]+str(rank)+'.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])

        
        #balancesheets______________________________________________
        #via pickle
        
        #creating a dictionary of firm dataframes
        #firmData={} defined in __init__
        colNames=["firm class type", "initial inventories","total costs", "revenuesCons", "revenuesInv", "consGoods inventories",\
       "invGoods inventories",  "consGoods in progr. inventories", "invGoods in progr. inventories", "profits", \
          "added value", "total production", "cost of production", "cost of unused factors", "total lost production", \
          "total cost of lost production", "cost of labor", "cost of capital", "capital quantity dynamic",\
            "production type"]
        

        for aFirm in context.agents(agent_type=0):
            self.firmData[aFirm.uid]=pd.DataFrame(aFirm.myBalancesheet)
            self.firmData[aFirm.uid].columns=colNames

        pickle.dump(self.firmData, open(params["log_file_root"]+'_balancesheetDict.p', "wb"))

        np.savetxt("plannerInfo.csv", context.agent((0,1,0)).informationTable, delimiter=",")

        print("cpu time - finishing phase", Tc(), "rank", rank, flush=True)
        print("THE END!", flush=True)
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

### if multi-rank, from the terminal launch: mpirun -n x ipython model1.0.ipynb  

where x is the number fo ranks

==========================================================================================================

In [9]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

["share of firms of that class, L min, L max, K min, K max, order duration min, order duration max, recipe, L prod, max order production, assets' useful life, planned markup, order observation frequency min, order observation frequency max, production type "]
[0.05, 50.0, 80.0, 6000.0, 9000.0, 5.0, 10.0, 120.0, 0.8, 100.0, 12.0, 0.3, 15.0, 20.0, 1.0]
[0.1, 30.0, 49.0, 3000.0, 4900.0, 5.0, 10.0, 100.0, 0.8, 70.0, 12.0, 0.3, 15.0, 20.0, 0.0]
[0.25, 10.0, 29.0, 800.0, 2320.0, 2.0, 4.0, 80.0, 0.7, 50.0, 12.0, 0.2, 10.0, 15.0, 0.0]
[0.4, 10.0, 29.0, 800.0, 2320.0, 1.0, 1.0, 50.0, 0.7, 100.0, 12.0, 0.3, 5.0, 10.0, 0.0]
[0.2, 2.0, 9.0, 100.0, 450.0, 1.0, 1.0, 50.0, 0.6, 10.0, 12.0, 0.1, 5.0, 10.0, 0.0]
rank 0 Initial price of durable productive goods per unit 1
rank 0 tick 0
rank 0 tick 10
rank 0 tick 20
rank 0 tick 30
rank 0 tick 40
rank 0 tick 50
rank 0 tick 60
rank 0 tick 70
rank 0 tick 80
rank 0 tick 90
rank 0 tick 100
rank 0 tick 110
rank 0 tick 120
rank 0 tick 130
rank 0 tick 140
rank 0